In [2]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from gensim.models import Word2Vec
import re

Varasemalt valiti välja teemad ja neid ilmestavad märksõnad manuaalselt, uurides süntaksianalüüsi tulemust. Märksõnade hulka võiks aga suurendada, et leida üles rohkem valitud teemasid käsitlevaid lauseid. Siin katsetatakse märksõnadele lisa leidmiseks Word2Vec mudelit. Word2Vec mudel treenitakse kõigi andmestikus olevate lausete peal, sest suuremal andmehulgal treenides võib oodata paremaid tulemusi.

### I Lausete sisselugemine andmebaasist

In [3]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [4]:
sentences = []

In [5]:
for row in cur.execute("SELECT sentence FROM sentences"):
    sentence = json_to_text(json_text=row[0])
    sentences.append(sentence)

In [6]:
con.close()

In [7]:
len(sentences)

89120

### II Lausete eeltöötlus

Enne Word2Vec mudelile etteandmist tuleb lauseid töödelda. See tähendab lausete väiketähestamist, erisümbolite eemaldamist ja lemmatiseerimist

In [8]:
def get_lemmas(sentence):
    lemma_list = []
    for i in range(len(sentence.words)):
        lemma_list.append(sentence.words[i].lemma[0])
    return lemma_list

In [9]:
lemmatized_sentences = [get_lemmas(sentence) for sentence in sentences]

In [10]:
lemmatized_sentences[0]

['tere',
 'siin',
 'kukkuma',
 'raadio',
 'keskpäevatund',
 'Tartu',
 'Rein',
 'kilk',
 'Tallinn',
 'Hans',
 'H',
 'luik',
 'ja',
 'priit',
 'hõbemägi',
 'ja',
 'see',
 'esimene',
 'lukk',
 'ka',
 'faktor',
 'olema',
 'lepik',
 'maa',
 'tõesti',
 'suur',
 'rahakott',
 'pühendama',
 'see',
 'kolm',
 'konteiner',
 'euroraha',
 'kolmene',
 'konteineritäis',
 '.']

In [11]:
lemmatized_sentences_joined = [" ".join(sentence) for sentence in lemmatized_sentences]

In [12]:
print(lemmatized_sentences_joined[0])

tere siin kukkuma raadio keskpäevatund Tartu Rein kilk Tallinn Hans H luik ja priit hõbemägi ja see esimene lukk ka faktor olema lepik maa tõesti suur rahakott pühendama see kolm konteiner euroraha kolmene konteineritäis .


In [19]:
lemmatized_sentences_lower = [sentence.lower() for sentence in lemmatized_sentences_joined]

In [20]:
print(lemmatized_sentences_lower[0])

tere siin kukkuma raadio keskpäevatund tartu rein kilk tallinn hans h luik ja priit hõbemägi ja see esimene lukk ka faktor olema lepik maa tõesti suur rahakott pühendama see kolm konteiner euroraha kolmene konteineritäis .


In [21]:
lemmatized_sentences_lower = [re.sub('\W', ' ', sentence) for sentence in lemmatized_sentences_lower]

In [22]:
print(lemmatized_sentences_lower[0])

tere siin kukkuma raadio keskpäevatund tartu rein kilk tallinn hans h luik ja priit hõbemägi ja see esimene lukk ka faktor olema lepik maa tõesti suur rahakott pühendama see kolm konteiner euroraha kolmene konteineritäis  


In [23]:
lemmatized_sentences_tokenized = [sentence.split() for sentence in lemmatized_sentences_lower]

In [24]:
print(lemmatized_sentences_tokenized[0])

['tere', 'siin', 'kukkuma', 'raadio', 'keskpäevatund', 'tartu', 'rein', 'kilk', 'tallinn', 'hans', 'h', 'luik', 'ja', 'priit', 'hõbemägi', 'ja', 'see', 'esimene', 'lukk', 'ka', 'faktor', 'olema', 'lepik', 'maa', 'tõesti', 'suur', 'rahakott', 'pühendama', 'see', 'kolm', 'konteiner', 'euroraha', 'kolmene', 'konteineritäis']


In [25]:
model = Word2Vec(lemmatized_sentences_tokenized, vector_size=100, window=2, min_count = 1,
                 seed = 1000, hs = 0, negative = 10, workers=16, epochs=20)

INFO:word2vec.py:582: collecting all words and their counts
INFO:word2vec.py:565: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:word2vec.py:565: PROGRESS: at sentence #10000, processed 253324 words, keeping 17519 word types
INFO:word2vec.py:565: PROGRESS: at sentence #20000, processed 482928 words, keeping 25985 word types
INFO:word2vec.py:565: PROGRESS: at sentence #30000, processed 719315 words, keeping 32960 word types
INFO:word2vec.py:565: PROGRESS: at sentence #40000, processed 1313189 words, keeping 43728 word types
INFO:word2vec.py:565: PROGRESS: at sentence #50000, processed 1632081 words, keeping 47861 word types
INFO:word2vec.py:565: PROGRESS: at sentence #60000, processed 1927637 words, keeping 51527 word types
INFO:word2vec.py:565: PROGRESS: at sentence #70000, processed 2215682 words, keeping 54795 word types
INFO:word2vec.py:565: PROGRESS: at sentence #80000, processed 2547985 words, keeping 59282 word types
INFO:word2vec.py:588: collected 63385 w

INFO:word2vec.py:1608: EPOCH 14 - PROGRESS: at 66.02% examples, 650767 words/s, in_qsize 31, out_qsize 1
INFO:word2vec.py:1652: EPOCH 14: training on 2870220 raw words (2001423 effective words) took 2.8s, 725306 effective words/s
INFO:word2vec.py:1608: EPOCH 15 - PROGRESS: at 40.03% examples, 728544 words/s, in_qsize 27, out_qsize 4
INFO:word2vec.py:1608: EPOCH 15 - PROGRESS: at 71.40% examples, 704848 words/s, in_qsize 31, out_qsize 0
INFO:word2vec.py:1652: EPOCH 15: training on 2870220 raw words (2002041 effective words) took 2.8s, 727063 effective words/s
INFO:word2vec.py:1608: EPOCH 16 - PROGRESS: at 40.31% examples, 731912 words/s, in_qsize 32, out_qsize 0
INFO:word2vec.py:1608: EPOCH 16 - PROGRESS: at 75.60% examples, 734010 words/s, in_qsize 32, out_qsize 0
INFO:word2vec.py:1652: EPOCH 16: training on 2870220 raw words (2001860 effective words) took 2.7s, 746273 effective words/s
INFO:word2vec.py:1608: EPOCH 17 - PROGRESS: at 38.62% examples, 630912 words/s, in_qsize 31, out_qsi

In [26]:
model.save("word2vec.model")

INFO:utils.py:448: Word2Vec lifecycle event {'fname_or_handle': 'word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-11-02T21:58:45.713565', 'gensim': '4.3.2', 'python': '3.10.11 | packaged by Anaconda, Inc. | (main, Apr 20 2023, 18:56:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'saving'}
INFO:utils.py:710: not storing attribute cum_table
INFO:utils.py:618: saved word2vec.model


In [27]:
model = Word2Vec.load("word2vec.model")

INFO:utils.py:482: loading Word2Vec object from word2vec.model
INFO:utils.py:516: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO:utils.py:554: setting ignored attribute cum_table to None
INFO:utils.py:448: Word2Vec lifecycle event {'fname': 'word2vec.model', 'datetime': '2023-11-02T21:58:48.743619', 'gensim': '4.3.2', 'python': '3.10.11 | packaged by Anaconda, Inc. | (main, Apr 20 2023, 18:56:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}


Teemad ja märksõnad:
1. Vaidlused ja erimeelsused: vastanduma, võitlema, häbistama, vaidlema
2. Valimised: kandideerima, valima, võitma, lubama, hääletama
3. Valetamine: valetama, varjama, mustama, kuritarvitama
4. Valitsemine: otsustama, kaasama, mõjutama, esindama, juhtima

In [28]:
model.wv.most_similar('vastanduma')
#tulemuses ei paista midagi sarnase tähendusega olevat

[('kokkupanija', 0.6224339008331299),
 ('rakkopp', 0.6214811205863953),
 ('toetajaskond', 0.6186890006065369),
 ('pealeriba', 0.6180562973022461),
 ('reformi', 0.6151947379112244),
 ('sisehall', 0.6084675192832947),
 ('auasi', 0.6069961190223694),
 ('kumuleeruma', 0.5954799056053162),
 ('esimehekandidaat', 0.5945754647254944),
 ('juhtfiguur', 0.5903006792068481)]

In [29]:
model.wv.most_similar('võitlema')
#saranase tähendusega 'protestima', 'sõdima'

[('protestima', 0.523410439491272),
 ('sõdima', 0.5013259649276733),
 ('seisma', 0.4866519570350647),
 ('töötama', 0.47069108486175537),
 ('vaidlema', 0.46901026368141174),
 ('vastutama', 0.46203529834747314),
 ('jubedasti', 0.44240596890449524),
 ('hoolitsema', 0.4423796236515045),
 ('paaritumine', 0.43765899538993835),
 ('lojaalne', 0.43651506304740906)]

In [30]:
model.wv.most_similar('häbistama')
#tulemuses ei paista midagi sarnase tähendusega olevat

[('isamaliit', 0.7511600255966187),
 ('kubermanguvalitsus', 0.7444968819618225),
 ('parteiladvik', 0.7417912483215332),
 ('mi5', 0.7394930720329285),
 ('vanaonutütar', 0.7384188771247864),
 ('harrastaja', 0.7380626797676086),
 ('uurimistoiming', 0.736935019493103),
 ('palkaja', 0.7369329333305359),
 ('sepatöö', 0.7353139519691467),
 ('jääkeldriaeg', 0.7334276437759399)]

In [31]:
model.wv.most_similar('vaidlema')
#sarnase tähendusega debateerima, diskuteerima, nurisema

[('protestima', 0.5775725245475769),
 ('arutlema', 0.5740605592727661),
 ('arutama', 0.5544575452804565),
 ('debateerima', 0.5212641954421997),
 ('diskuteerima', 0.49965736269950867),
 ('nurisema', 0.48652660846710205),
 ('hääletama', 0.47399431467056274),
 ('netisait', 0.4731456935405731),
 ('võitlema', 0.46901026368141174),
 ('poolmaraton', 0.4589269757270813)]

In [32]:
model.wv.most_similar('kandideerima')
#sarnase tähendusega 'osalema'

[('valima', 0.6270256042480469),
 ('asendusliige', 0.589776337146759),
 ('üksikkandidaat', 0.5729894042015076),
 ('aseesimees', 0.559035062789917),
 ('valitud', 0.5348803400993347),
 ('linnapea', 0.5284157991409302),
 ('hääletama', 0.5265281200408936),
 ('valimiskogu', 0.524062991142273),
 ('kandidaat', 0.5151594877243042),
 ('osalema', 0.5128973126411438)]

In [33]:
model.wv.most_similar('valima')
#sarnase tähendusega sõnu on leitud, kuid need on juba märksõnade seas

[('kandideerima', 0.6270256042480469),
 ('valitud', 0.6044657826423645),
 ('otsustama', 0.5826196074485779),
 ('hääletama', 0.5817234516143799),
 ('valimiskogu', 0.5384629368782043),
 ('eelistama', 0.5249647498130798),
 ('kuratoorium', 0.48315897583961487),
 ('määrama', 0.47138163447380066),
 ('mõtlema', 0.4658842384815216),
 ('valimine', 0.45968955755233765)]

In [34]:
model.wv.most_similar('võitma')
#sarnase tähendusega sõnu on leitud, kuid need on juba märksõnade hulgas

[('osalema', 0.5104299783706665),
 ('kaotama', 0.49529433250427246),
 ('hinnatuim', 0.46480485796928406),
 ('sõltuma', 0.4645591080188751),
 ('kandideerima', 0.4616854786872864),
 ('verikogu', 0.45450359582901),
 ('valituskogu', 0.439203679561615),
 ('eel', 0.4341794550418854),
 ('pakkuma', 0.4307640492916107),
 ('valima', 0.42971187829971313)]

In [35]:
model.wv.most_similar('lubama')
#sarnase tähendusega 'kavatsema'

[('kavatsema', 0.5568326115608215),
 ('tahtma', 0.5117815732955933),
 ('registreerima', 0.4993237853050232),
 ('kinnitama', 0.4914655089378357),
 ('saama', 0.4732564091682434),
 ('otsustama', 0.46696409583091736),
 ('soovima', 0.4660798907279968),
 ('küsitud', 0.46385112404823303),
 ('ostma', 0.4625377357006073),
 ('võimus', 0.4596814513206482)]

In [36]:
model.wv.most_similar('hääletama')
#sarnase tähendusega 'toetama'

[('valima', 0.5817233920097351),
 ('valitud', 0.5272095799446106),
 ('kandideerima', 0.5265281200408936),
 ('valimiskogu', 0.5229399800300598),
 ('protestima', 0.5199283361434937),
 ('mandaat', 0.4970180094242096),
 ('riigikogu', 0.49058881402015686),
 ('otsustama', 0.4866863191127777),
 ('vaidlema', 0.47399425506591797),
 ('toetama', 0.45921438932418823)]

In [37]:
model.wv.most_similar('valetama')
#sarnase tähendusega 'manipuleerima', kuid 'ründama' ja 'süüdistama' sobiks vaidluste ja erimeelsuste hulka

[('ründama', 0.5853550434112549),
 ('süüdistama', 0.5348684191703796),
 ('usaldama', 0.5113126039505005),
 ('mõistma', 0.5027973055839539),
 ('manipuleerima', 0.4990549683570862),
 ('rahuldama', 0.4936761260032654),
 ('lekitama', 0.4896601736545563),
 ('taipama', 0.48760750889778137),
 ('tajuma', 0.47582074999809265),
 ('oskama', 0.4639304578304291)]

In [38]:
model.wv.most_similar('varjama')
#sobivaid sõnu ei leitud

[('pistmine', 0.5733086466789246),
 ('taipama', 0.5644692182540894),
 ('kobisema', 0.5371357798576355),
 ('takistama', 0.5133538246154785),
 ('põhjendama', 0.5128811597824097),
 ('reklaamima', 0.5063628554344177),
 ('krussis', 0.5027134418487549),
 ('süüdistama', 0.4951495826244354),
 ('taluma', 0.4872486889362335),
 ('parata', 0.4788810610771179)]

In [39]:
model.wv.most_similar('mustama')
#sobivaid sõnu ei leitud

[('honda', 0.7582514882087708),
 ('teedeehituskontsern', 0.7538400888442993),
 ('rüü', 0.7451639175415039),
 ('soomustama', 0.7389765381813049),
 ('daugava', 0.7351694107055664),
 ('avatar', 0.7349169850349426),
 ('võsane', 0.7321881055831909),
 ('totterdava', 0.7292080521583557),
 ('very', 0.7258449196815491),
 ('käesjaam', 0.7257856130599976)]

In [40]:
model.wv.most_similar('kuritarvitama')
#sobivaid sõnu ei leitud

[('saatetiir', 0.696429967880249),
 ('vasakpopulist', 0.671026349067688),
 ('setukas', 0.6692653298377991),
 ('warmale', 0.6643701195716858),
 ('munitsipaalväljaanne', 0.6549315452575684),
 ('usurpeerinu', 0.6501203179359436),
 ('elektrilas', 0.6495986580848694),
 ('ühistegemine', 0.646170973777771),
 ('regress', 0.6461708545684814),
 ('kodusool', 0.6416218280792236)]

In [41]:
model.wv.most_similar('otsustama')
#sarnase tähendusega sõnad 'määrama', 'kinnitama', 'menetlema' ning võibolla ka 'vastutama'

[('valima', 0.5826196670532227),
 ('määrama', 0.5304931998252869),
 ('menetlema', 0.5071468353271484),
 ('kinnitama', 0.5063421726226807),
 ('vastutama', 0.49241870641708374),
 ('hääletama', 0.4866863489151001),
 ('otsus', 0.4860902428627014),
 ('küsima', 0.4769878089427948),
 ('registreerima', 0.47688907384872437),
 ('veenma', 0.47455453872680664)]

In [42]:
model.wv.most_similar('kaasama')
#saranse tähendusega sõnad 'suunama', 'paigutama', aga võibolla ka 'finantseerima', 'investeerima' ja 'rahastama'

[('finantseerima', 0.5306695103645325),
 ('investeerima', 0.5123718976974487),
 ('suunama', 0.5005371570587158),
 ('paigutama', 0.4950893223285675),
 ('hankima', 0.46612435579299927),
 ('usaldama', 0.45946404337882996),
 ('manipuleerima', 0.45832550525665283),
 ('rahastama', 0.44900715351104736),
 ('põhjendama', 0.4466596245765686),
 ('aktsepteerima', 0.44045642018318176)]

In [43]:
model.wv.most_similar('mõjutama')
#saranase tähendusega sõnad 'mõjuma', 'piirama'

[('sõltuma', 0.6637705564498901),
 ('mõjuma', 0.6015416979789734),
 ('kajastuma', 0.5434731841087341),
 ('kajastama', 0.506058394908905),
 ('kahjustama', 0.5025191307067871),
 ('piirama', 0.5001704096794128),
 ('edendama', 0.48526307940483093),
 ('puudutama', 0.4767216444015503),
 ('arenema', 0.47497254610061646),
 ('jälgima', 0.47483912110328674)]

In [44]:
model.wv.most_similar('esindama')
#sobivaid sõnu ei leitud

[('esindaja', 0.5250771045684814),
 ('kuuluv', 0.5179451107978821),
 ('huvituma', 0.4994533658027649),
 ('delegatsioon', 0.4755462408065796),
 ('lojaalne', 0.4745830297470093),
 ('vestlema', 0.47226694226264954),
 ('suhtlema', 0.4703591763973236),
 ('liige', 0.46780064702033997),
 ('eelistama', 0.46206408739089966),
 ('kuuluma', 0.4582160711288452)]

In [45]:
model.wv.most_similar('juhtima')
#sobivaid sõnu ei leitud

[('juhitud', 0.6343020796775818),
 ('juhtinud', 0.5932800769805908),
 ('juhtimine', 0.5384437441825867),
 ('pälvima', 0.5209156274795532),
 ('maajuhiks', 0.5043453574180603),
 ('paeluma', 0.5033291578292847),
 ('pöör', 0.4989897906780243),
 ('pöörata', 0.4976150393486023),
 ('pöörama', 0.4900239408016205),
 ('koonduma', 0.489751935005188)]

Sõnad, mis sobivatena tunduvad, lisatakse manuaalselt teemade ja märksõnade nimekirja.